* handle baby funcs and user funcs
* sometimes v is a Value sometimes it's just given


* VectorizeUserFunc? What?
* wtf is up with Filter
* Reduce vs ReduceSeq

Ops:
    - Get = ._
    - Value : val, type
    VectorizeUserFun : function, arg
    Transpose : arg
    Join : arg
    Zip : arg, arg
    Map : function, arg
    Filter : ?
    ReduceSeq : ?
    Reduce : function, val, arg
    Lambda : fun(...) | \(...)
    - Params
    UserFuncs

./Debug/bin/kiter -f <file> -a PrintInfos

* strip quotes from function?
* what's the point of \?
* handle getting el from tuple (Get)
* :>> ??? = pipe in to next op I think
* 0 vs 0.0f vs "0.0f" vs 1 vs 3.40282347e+38

Ops:
    - Get = ._
    - Value : val, type
    VectorizeUserFun : function, arg
    Transpose : arg
    Join : arg
    Zip : arg, arg
    Map : function, arg
    Filter : ?
    ReduceSeq : ?
    Reduce : function, val, arg
    Lambda : fun(...) | \(...)
    - Params
    UserFuncs

Piping:
    o : apply - f o g(x) = f(g(x))
    \$ : piping  - f \$ x = f(x)
    :>> : reverse piping - x :>> f = f(x)

Types:
 'TupleType',
 'Float4',
 'Int',
 'Float',
 'ArrayType',

In [1]:
import os, re, random
# from lxml import etree

def flatten(lol):
    return [val for sublist in lol for val in sublist]
def printls(ls):
    for item in ls:
        print(item)
        
def getcn(thingy):
    return thingy.__class__.__name__

program_names = []
programs = []
def load_programs():
    liftdir = "/Users/hebe/Dropbox/Yale-NUS/Yr4/capstone/forrealsies/lift"
    global program_names
    program_names += os.listdir(liftdir + "/highLevel")
    program_names = [x for x in program_names if 
                os.path.isfile(os.path.join(liftdir,"highLevel",x)) and ('.' not in x)]

    for program_name in program_names:
        with open(os.path.join(liftdir,"highLevel",program_name)) as f:
            programs.append(f.read())

In [2]:
# Input: string
# Output: tuple: (string before brackets, 
#                 top-level contents of first brackets (or full string if no brackets),
#                 string after brackets)
# Error state: If brackets not closed
def inbc(s):
    if '(' not in s:
        return "",s,""
    start = s.index('(')
    open_ctr = 0
    for i in range(start+1, len(s)):
        if s[i] == '(':
            open_ctr += 1
        if s[i] == ')':
            if open_ctr == 0:
                return s[0:start], s[start+1:i], s[i+1:]
            else:
                open_ctr -= 1
    return False

def getcbc(s):
    if '{' not in s:
        return s
    start = s.index('{')
    open_ctr = 0
    for i in range(start+1, len(s)):
        if s[i] == '{':
            open_ctr += 1
        if s[i] == '}':
            if open_ctr == 0:
                return s[start+1:i]
            else:
                open_ctr -= 1
    return False

def getbc(s):
    return inbc(s)[1]

def inbc_rev(s):
    if '(' not in s:
        return "",s,""
    end = s.rindex(')')
    open_ctr = 0
    for i in range(end-1, -1, -1):
        if s[i] == ')':
            open_ctr += 1
        if s[i] == '(':
            if open_ctr == 0:
                return s[0:i], s[i+1:end], s[end+1:]
            else:
                open_ctr -= 1
    return False    

# Input: Overarching program
# Output: Main function
def getfunc(s):
    start = s.index('\nfun')
    s = getbc(s[start:])
    return re.sub(r"([a-zA-Z0-9]+)._([0-9]+)", r"Get(\g<1>, \g<2>)",s)

# Input: Overarching program
# Output: List of all SizeVar names
def getsvs(s):
    return re.findall(r'val\s+([a-zA-Z]+)\s*=\s*SizeVar\("[a-zA-Z]+"\)', s)

# Input: function
# Output: body of function
def getbody(s):
    i = s.index("=>")
    s = s[i+2:].strip()
    if s[0] == '(':
        s = getbc(s)
    if s[0] == '{':
        s = getcbc(s)
    if s[0] == '(':
        s = getbc(s)
    return s
def getmainbody(s):
    s = getfunc(s)
    return getbody(s)

In [3]:
# Input: string of comma-separated items
# Output: list of top-level items
def commasplit(s):
    commas = [-1]
    open_ctr = 0
    for i in range(len(s)):
        if s[i] in ['(', '{', '[']:
            open_ctr += 1
        if s[i] in [')', '}', ']']:
            open_ctr -= 1
        if s[i] == ',' and open_ctr == 0:
            commas.append(i)
    commas.append(len(s))
    broken = [s[commas[j-1]+1:commas[j]] for j in range(1, len(commas))]
    return [x.strip() for x in broken if len(x.strip()) > 0]

# Input: string of comma-separated items
# Output: list of top-level items
def commasplit_rev(s):
    commas = [len(s)]
    open_ctr = 0
    for i in range(len(s)-1 , -1, -1):
        if s[i] in [')', '}', ']']:
            open_ctr += 1
        if s[i] in ['(', '{', '[']:
            open_ctr -= 1
        if s[i] == ',' and open_ctr == 0:
            commas.append(i)
    commas.append(-1)
    commas.reverse()
    broken = [s[commas[j-1]+1:commas[j]] for j in range(1, len(commas))]
    return [x.strip() for x in broken if len(x.strip()) > 0]

# Input: Main function with input types, input names, and function
# Output: list of pairs (input name, input type)
def getinputs(s):
    f = s.split("=>")[0]
    f = commasplit_rev(f)
    t = f[0:-1]
    n = getbc(f[-1])
    n = commasplit_rev(n)
    return [(a.strip(), b.strip()) for a, b in zip(n,t)]
def getmaininputs(s):
    return getinputs(getfunc(s))


# Input: lambda function
# Output: names of paramaters
def getnames(s):
    f = s.split("=>")[0]
    ns = getbc(f)
    return commasplit(ns)


# Input: Overarching program
# Output: Given subfunctions used in main function
def getsubfuncs(s):
    sfnames = re.findall(r"\ndef +([a-zA-Z0-9]+) *= *fun", s)
    return [(name, getbc(s[s.index(name):])) for name in sfnames]
def getsplit(s):
    return [x for x in re.split(r"[\(\){},\s]", s) if len(x) > 0]

def getNextOp(s):
    s = s.strip()
    return re.match(r"([A-Za-z0-9]+)", s).groups()[0]

def getNextParams(s):
    return commasplit_rev(getbc(s))
def hasparams(s):
    return '(' in s or '{' in s

def isfunction(s):
    if s.strip().startswith(("\(", "fun(")):
        return True
    return False

# Input: body of function
# Output: list of top-level functions separated by operators
def componentsplit(s):
    commas = [0]
    open_ctr = 0
    for i in range(len(s)):
        if s[i] in ['(', '{', '[']:
            open_ctr += 1
        if s[i] in [')', '}', ']']:
            open_ctr -= 1
        if open_ctr == 0:
            if s[i:].startswith("$"):
                commas += [i, i + 1]
            if s[i:].startswith(":>>"):
                commas += [i, i + 3]
            if re.match(r".\bo\b", s[i:]):
                commas += [i+1, i + 2]
    commas.append(len(s))
#     print(commas)
    broken = [s[commas[j-1]:commas[j]].strip() for j in range(1, len(commas))]
    return broken

In [34]:
tst = Map('k',1)
print(tst.has_dt())
tst.add_dt(1)
tst.get_dt()
print(tst.has_dt())


False
True


In [4]:
class Channel:
    def __init__ (self, src_act, dst_act,  src, dst, init_token = 0):
        self.name = src+"_to_"+dst
        self.src_port = src
        self.src_act = src_act
        self.dst_port = dst
        self.dst_act = dst_act
        self.init_token = init_token
        self.size = 1
    def __str__(self):
        return "<channel name='"+self.name+"' srcActor='"+self.src_act+"' srcPort='"+self.src_port+"' dstActor='"+self.dst_act+"' dstPort='"+self.dst_port+"' size='"+str(self.size)+"' initialTokens='"+str(self.init_token)+"'/>"


class Port:
    def __init__ (self, direction, name, rate):
        self.name = name
        self.direction = direction
        self.rate = rate
    def __str__(self):
        return "<port type='"+self.direction+"' name='"+self.name+"' rate='"+','.join(map(str,self.rate))+"'/>"

def get_channel(src, dst):
    return Channel(src.name.split("_")[0], dst.name.split("_")[0], src.name, dst.name)
# INPUT: list of dicts of ports - [{'dst' : Port, 'src' : Port}]
# OUTPUT: list of channels - [Channel]
def get_channels(ls):
    return [get_channel(x["src"], x["dst"]) for x in ls]

class CSDFNode:
    def __init__():
        pass
    def add_dt(self, dt):
        self.datatype = dt
    def get_dt(self):
        return self.datatype
    def has_dt(self):
        try:
            if self.datatype == True:
                return True
            else:
                return True
        except:
            return False
        
class Param(CSDFNode):
    def __init__(self, name, label):
        self.name = name
        self.label = label
        self.input = Port('in', name + '_in', [1])
        self.out_ctr = 0
        self.output = []
    def new_outport(self, r = [1]):
        np = Port('out', self.name + "_x"+str(self.out_ctr)+ '_out', r)
        self.output.append(np)
        self.out_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Param' >\n" + '\n'.join(map(str, [self.input]+self.output)) + "\n</actor>"

class Get(CSDFNode):
    def __init__(self, name, idx):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
        self.idx = idx
    def __str__(self):
        return "<actor name='"+self.name+"' type='Get' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

class Transpose(CSDFNode):
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Transpose' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

    
class Dearray(CSDFNode):
    def __init__(self, name, portin, portout):
        self.name = name
        self.input = portin
        self.output = portout
    def __str__(self):
        return "<actor name='"+self.name+"' type='Dearray' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    

        
class Rearray(CSDFNode):
    def __init__(self, name, portin, portout, mstr):
        self.name = name
        self.input = portin
        self.output = portout
        self.masternode = mstr
    def __str__(self):
        return "<actor name='"+self.name+"' type='Rearray' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    

    
class Join(CSDFNode):
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Join' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    
# Must have new value for every attachment
class Value(CSDFNode):
    def __init__(self, name, value, kind):
        self.name = name
        self.value = value
        self.kind = kind
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Value' value='"+self.value+"' kind='"+self.kind+"'>\n" + '\n'.join(map(str, self.output)) + "\n</actor>"

class Reduce(CSDFNode):
    def __init__(self, name, subfunc, initval, seq = False):
        self.name = name
        self.type =  "ReduceSeq" if seq else "Reduce"
        self.subfunc = subfunc
        self.initval = initval
        self.seq = seq
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='"+self.type+"'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"


class Map(CSDFNode):
    def __init__(self, name, subfunc):
        self.name = name
        self.subfunc = subfunc
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Map'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"
    
class Zip(CSDFNode):
    def __init__(self, name : str):
        self.name = name
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Zip'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

class Mather(CSDFNode):
    def __init__(self, name, kind):
        self.name = name
        self.kind = kind
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='"+self.kind+"'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

class UserFun(CSDFNode):
    def __init__(self, name : str, label):
        self.name = name
        self.label = label
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='UserFun'  label='"+self.label+"'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

def getports(node):
    return (node.input if type(node.input) == list else [node.input]) + (node.output if type(node.output) == list else [node.output])
    

In [5]:
def unpack_program(program):
    node_ctr = 0
    def getname():
        nonlocal node_ctr
        node_ctr += 1
        return "n" + str(node_ctr)
    
    named_nodes = {}
    
    starting_inputs = getmaininputs(program)
    sizevars = getsvs(program)
    main_func = commasplit(getfunc(program))[-1]
    
    def mk_bby_func(bby, n_inputs):
        assert(len(set(bby.strip()).intersection(set("{}()[],. "))) == 0)
        if bby in ["mult", "add"]:
            new_node = Mather(getname(), bby)
            return {
                "nodes" : [new_node],
                "channels" : [],
                "inputs" : [new_node.new_inport(), new_node.new_inport()],
                "output" : new_node.output
            }
            
        new_node = UserFun(getname(), bby)
        for i in range(n_inputs):
            new_node.new_inport()
        return {
            "nodes" : [new_node],
            "channels" : [],
            "inputs" : new_node.input,
            "output" : new_node.output
        }
        
    
    # Output: {nodes : [Node], channels : [{Port, Port}], inputs : [Port], output : Port}
    def unpack_func(func):
        nodes = []
        channels = []
        myinputs = []
        local_names = getnames(func) if "=>" in func else []

        for name in local_names:
            new_node = Param(getname(), name)
            nodes.append(new_node)
            myinputs.append(new_node.input)
            assert(name not in named_nodes)
            named_nodes[name] = new_node  
        
        # Input: body
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_body(body):
            comps = componentsplit(body)
            assert(len(comps) > 0)
            assert(len(comps) % 2 == 1)
            
            curr = unpack_comp(comps[0])
            for i in range(1, len(comps), 2):
                assert(comps[i] in ["o", "$", ":>>"])
                new = unpack_comp(comps[i+1])
                if comps[i] == ":>>":
                    assert(new["in"] and curr["out"])
                    channels.append({"dst":new["in"], "src" : curr["out"]})
                    curr["out"] = new.get("out")
                else:
                    assert(curr["in"] and new["out"])
                    channels.append({"dst" : curr["in"], "src" : new["out"]})
                    curr["in"] = new.get("in")
            return curr
            
        
        # Input: single component
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_comp(comp):
            assert(len(componentsplit(comp)) == 1)
            op = getNextOp(comp)
            if op in named_nodes.keys():
                assert(op == comp.strip())
                src = named_nodes[op]
                src_port = src.new_outport()
                return({"out" : src_port})
            if op == "Value":
                args = getNextParams(comp)
                # It's the final node
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Value(getname(), args[0], args[1])
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Get":
                args = getNextParams(comp)
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Get(getname(), args[1])
                incoming = unpack_body(args[0])["out"]
                channels.append({"src" : incoming, "dst" : new_node.input})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Transpose":
                assert(len(getNextParams(comp)) == 0)
                new_node = Transpose(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Join":
                assert(len(getNextParams(comp)) == 0)
                new_node = Join(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Zip":
                args = getNextParams(comp)
                new_node = Zip(getname())
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op in ["mult", "add"]:
                args = getNextParams(comp)
                new_node = Mather(getname(), op)
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Map":
                args = getNextParams(comp)
                assert(len(args) == 1)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 1)
                new_node = Map(getname(), child_func)
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op in ["Reduce", "ReduceSeq"]:
                args = getNextParams(comp)
                assert(len(args) == 2)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 2)
                init_val = args[1]
                new_node = Reduce(getname(), child_func, init_val, op == "ReduceSeq")
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            else:
                new_node = UserFun(getname(), op)
                if hasparams(comp):
                    for arg in getNextParams(comp):
                        new_in = new_node.new_inport()
                        new_out = unpack_body(arg)["out"]
                        channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
        
        
        mybod = getbody(func) if "=>" in func else func
        res = unpack_body(mybod)
        if "=>" in func:
            assert(not res.get("in"))
        else:
            myinputs = [res["in"]]
        myoutput = res["out"]
        
        for name in local_names:
            named_nodes.pop(name)
        
        return {
            "nodes" : nodes,
            "channels" : get_channels(channels),
            "inputs" : myinputs,
            "output" : myoutput
        }
            
    parsed = unpack_func(main_func)
    return {
        "sizevars" : sizevars,
        "inputs" : starting_inputs,
        "code" : getfunc(program),
        "graph" : parsed
    }       

In [6]:

def printgraph(g):
    n = g["nodes"]
    c = g["channels"]
    printls(n)
    printls(c)
    
    for node in n:
        if getcn(node).startswith(("Reduce", "Map", "Iterate")):
            print("\n       Subgraph of", getcn(node),"node ", node.name)
            printgraph(node.subfunc)

In [40]:
def get_node_by_port(nodes, port_name):
    ns = [n for n in nodes if any([p.name == port_name for p in getports(n)])]
    assert(len(ns) == 1)
    return ns[0]

def get_connected_node(g, port_name):
    nodes = g['nodes']
    channels = g['channels']
    channel = [c for c in channels if c.src_port == port_name or c.dst_port == port_name]
    if len(channel) == 0:
        return None
    assert(len(channel) == 1)
    channel = channel[0]
    other_port_name = channel.dst_port if channel.src_port == port_name else channel.src_port 
    return get_node_by_port(nodes, other_port_name)

In [8]:
def explode(graph):
    def_len = 10
    nodes = graph["nodes"]
    channels = graph["channels"]
    
    # get Map nodes
    mapnodes = [n for n in nodes if getcn(n) == "Map"]
    # explode Map nodes
    for node in mapnodes:
        name = node.name
        da_in = node.input
        da_out = Port("out", name + "XDA_out", [def_len])
        da = Dearray(name + "XDA", da_in, da_out)
        ra_out = node.output
        ra_in = Port("in", name + "XRA_in", [def_len])
        ra = Rearray(name + "XRA", ra_in, ra_out, "Map")
        
        subgraph = explode(node.subfunc)
        nodes += subgraph["nodes"]
        nodes += [da, ra]
        
        old_in_channel = [c for c in channels if c.dst_act == name]
        old_out_channel = [c for c in channels if c.src_act == name]
        assert(len(old_in_channel) <= 1)
        assert(len(old_out_channel) <= 1)
        if len(old_in_channel) == 1:
            old_in_channel[0].dst_act = da.name
        if len(old_out_channel) == 1:
            old_out_channel[0].src_act = ra.name
        channels += subgraph["channels"]
        so = subgraph["output"].name
        si = subgraph["inputs"][0].name
        channels += [Channel(da.name, 
                             get_node_by_port(subgraph["nodes"], si).name, 
                             da_out.name, si), 
                     Channel(get_node_by_port(subgraph["nodes"], so).name, 
                             ra.name, so, ra_in.name)]
    # remove Map nodes
    nodes = [n for n in nodes if getcn(n) != "Map"]
    
    
    # get Reduce nodes
    reducenodes = [n for n in nodes if getcn(n).startswith("Reduce")]
    # explode Reduce nodes
    for node in reducenodes:
        name = node.name
        da_in = node.input
        da_out = Port("out", name + "XDA_out", [def_len])
        da = Dearray(name + "XDA", da_in, da_out)
        ra_in = Port("in", name + "XRA_in", [1])
        ra_out = node.output
        ra = Rearray(name + "XRA", ra_in, ra_out, "Reduce")
        
        # make param for recursion
        para = Param(name + "XPARA", "recursion")
        para.input.rate = [1] * def_len
        para_out_0 = para.new_outport([1] * def_len)
        para_out_1 = para.new_outport([0] * (def_len - 1) + [1])
        
        # add nodes
        subgraph = explode(node.subfunc)
        nodes += subgraph["nodes"]
        nodes += [da, ra, para]
        
        # add channels
        old_in_channel = [c for c in channels if c.dst_act == name]
        old_out_channel = [c for c in channels if c.src_act == name]
#         printls(old_in_channel)
#         printls(old_out_channel)
        if len(old_in_channel) == 1:
            old_in_channel[0].dst_act = da.name
        if len(old_out_channel) == 1:
            old_out_channel[0].src_act = ra.name
        
        si = subgraph["inputs"]
        so = subgraph["output"].name
        channels += [Channel(para.name, get_node_by_port(subgraph["nodes"], si[1].name).name, 
                             para_out_0.name, si[1].name, 1),
                     Channel(da.name, get_node_by_port(subgraph["nodes"], si[0].name).name, 
                             da_out.name, si[0].name),
                     Channel(get_node_by_port(subgraph["nodes"], so).name, para.name,
                             so, para.input.name),
                     Channel(para.name, ra.name,
                             para_out_1.name, ra_in.name)]
        channels += subgraph["channels"]
        
    # remove Reduce nodes
    nodes = [n for n in nodes if not getcn(n).startswith("Reduce")]
    
    
    graph["nodes"] = nodes
    graph["channels"] = channels
    return graph
            

In [9]:
def getin(n):
    if type(n.input) is list:
        return n.input
    else:
        return [n.input]
def getout(n):
    if type(n.output) is list:
        return n.output
    else:
        return [n.output]

def check_correct(g):
    ns = g["nodes"]
    cs = g["channels"]
    nd = dict([(n.name, n) for n in ns])
    
    for c in cs:
        assert(c.dst_port in [p.name for p in getin(nd[c.dst_act])])
        assert(c.src_port in [p.name for p in getout(nd[c.src_act])])

In [10]:
def smush_rede(g):
    ns = g["nodes"]
    cs = g["channels"]
    n_dict = dict([(n.name, n) for n in ns])
    rds = [ c for c in cs if ((getcn(n_dict[c.src_act]) == "Dearray" 
                        and getcn(n_dict[c.dst_act]) == "Rearray")
                       or (getcn(n_dict[c.src_act]) == "Rearray" 
                        and getcn(n_dict[c.dst_act]) == "Dearray"))]
    for rd in rds:
        src = n_dict[rd.src_act]
        dst = n_dict[rd.dst_act]
        assert(getcn(src) in ["Rearray", "Dearray"])
        assert(getcn(dst) in ["Rearray", "Dearray"])
        if src.output.rate == dst.input.rate:
            inc_c = [c for c in cs if  c.dst_port == src.input.name][0]
            out_c = [c for c in cs if c.src_port == dst.output.name][0]
            new_c = Channel(inc_c.src_act, out_c.dst_act, inc_c.src_port, out_c.dst_port)
            cs = [c for c in cs if c not in [inc_c, out_c, rd]] + [new_c]
            ns = [n for n in ns if n not in [src, dst]]
    g["nodes"] = ns
    g["channels"] = cs
    
    return g

In [11]:
xml_hdr = """<?xml version="1.0" encoding="UTF-8"?>
<sdf3 type="csdf" version="1.0"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:noNamespaceSchemaLocation="http://www.es.ele.tue.nl/sdf3/xsd/sdf3-csdf.xsd">
<applicationGraph name='autogen'>
    <csdf name='autogen' type='autogen'>
    """

xml_mid = """
    </csdf>

    <csdfProperties>
    """

xml_end = """
    </csdfProperties>

</applicationGraph>

</sdf3>

"""

def check_and_get_phasecount(node):
    ports = getports(node)
    phase_counts = [len(p.rate) for p in ports]
    assert(len(set(phase_counts)) == 1)
    return phase_counts[0]

def get_actor_properties(node):
    ext = 4
    if getcn(node) in ["Param", "Value", "Get"]:
        ext = 1
    if getcn(node) in ["Split", "Join", "Dearray", "Rearray", "Zip"]:
        ext = 3
    if getcn(node) in ["Transpose", "UserFun"]:
        ext = 10
    phase_count = str(ext)+(',' + str(ext)) * (check_and_get_phasecount(node) - 1)
    properties = """<actorProperties actor='{0}'>
        <processor type='cluster_0' default='true'>
            <executionTime time='{1}'/>
        </processor>
    </actorProperties>
    """.format(node.name, phase_count)
    return properties

def get_sdf(g):
    nodes = g["nodes"]
    channels = g["channels"]
    
    # Build
    sdf = xml_hdr
    sdf += "\n".join([str(node) for node in nodes])
    sdf += "\n".join([str(channel) for channel in channels])
    sdf += xml_mid
    sdf += "\n".join([get_actor_properties(node) for node in nodes])
    sdf += xml_end
    
    # Prettify
#     root = etree.fromstring(sdf)
#     sdf = etree.tostring(root, pretty_print=True).decode()
    
    return sdf
    
    
def get_sdf_from_hl(p):
    return get_sdf(smush_rede(explode(unpack_program(p)["graph"])))

In [12]:
def batch_read_and_write(source_dir, target_dir):
    program_names = os.listdir(source_dir)
    program_names = [x for x in program_names if 
                os.path.isfile(os.path.join(source_dir, x)) and ('.' not in x)]
    
    for program_name in program_names:
        with open(os.path.join(source_dir,program_name)) as f:
            program = f.read()
        sdf = get_sdf_from_hl(program)
        with open(os.path.join(target_dir,program_name+".txt"), "w") as f:
            f.write(sdf)

In [13]:

source_dir = "/Users/hebe/Dropbox/Yale-NUS/Yr4/capstone/forrealsies/lift/highLevel"
target_dir = "/Users/hebe/Dropbox/Yale-NUS/Yr4/capstone/helpers/sdfxmls"
batch_read_and_write(source_dir, target_dir)

In [14]:
load_programs()

parsed_programs =[unpack_program(p) for p in programs]
for p in parsed_programs:
    ex = explode(p["graph"])
    ex = smush_rede(ex)
    check_correct(ex)
    p["exploded"] = ex
sdfs = [get_sdf_from_hl(p) for p in programs]


In [15]:
pp = parsed_programs[0]
gg = pp["exploded"]
nn = gg["nodes"]
cc = gg["channels"]
nd = dict([(n.name, n) for n in nn])

In [16]:
program_names

['gemvN',
 'mmNN',
 'dot',
 'mmNT',
 'nbody',
 'gesummvNN',
 'gemvT',
 'molecularDynamics',
 'mriqComputeQ',
 'mandelbrot',
 'blackScholes',
 'mriqPhiMag',
 'asum',
 'mv',
 'nearestNeighbour',
 'mmTT',
 'scal',
 'mvAsMM',
 'kmeans',
 'gesummvTT',
 'mmTN']

In [17]:
gemvt = parsed_programs[6]

In [18]:
p = gemvt["code"]
print(p)


  ArrayType(ArrayType(Float, N), M),
  ArrayType(Float, M),
  ArrayType(Float, N),
  Float,
  Float,
  (matrix, vectorX, vectorY, alpha, beta) => {
    Map(fun(t =>
      Map(fun(x =>
        add(
          mult(x, alpha),
          mult(Get(t, 1), beta)
        )
      )) o
        Reduce(add, 0.0f) o
        Map(fun(x => mult(Get(x, 0), Get(x, 1)))) $ Zip(vectorX, Get(t, 0))
    )) $ Zip(Transpose() $ matrix, vectorY)
  }


In [19]:
dsvs = [34,40,10,50,5,3]

nts = getmaininputs(programs[0])
sv = getsvs(programs[0])

In [20]:

def printgraph_simple(g):
    n = g["nodes"]
    c = g["channels"]
    printls([node.name + ": " + getcn(node) for node in n])
    printls([(channel.src_act, channel.dst_act)] for channel in c)
    
    for node in n:
        if getcn(node).startswith(("Reduce", "Map", "Iterate")):
            print("\n       Subgraph of", getcn(node),"node ", node.name)
            printgraph_simple(node.subfunc)

In [21]:

printgraph_simple(parsed_programs[1]["graph"])

n1: Param
n2: Param
n3: Param
n14: Transpose
n4: Param
n12: Zip
n7: Param
n9: Get
n10: Get
n8: Mather
n11XDA: Dearray
n5: Mather
n6XRA: Rearray
n6XPARA: Param
n13XDA: Dearray
n13XRA: Rearray
n15XDA: Dearray
n15XRA: Rearray
[('n1', 'n15XDA')]
[('n14', 'n13XDA')]
[('n2', 'n14')]
[('n3', 'n12')]
[('n4', 'n12')]
[('n12', 'n11XDA')]
[('n7', 'n9')]
[('n9', 'n8')]
[('n7', 'n10')]
[('n10', 'n8')]
[('n11XDA', 'n7')]
[('n6XPARA', 'n5')]
[('n5', 'n6XPARA')]
[('n6XPARA', 'n6XRA')]
[('n13XDA', 'n4')]
[('n6XRA', 'n13XRA')]
[('n15XDA', 'n3')]
[('n13XRA', 'n15XRA')]
[('n8', 'n5')]


In [22]:
k = unpack_program(programs[1])

printgraph_simple(k["graph"])

n1: Param
n2: Param
n15: Map
[('n1', 'n15')]

       Subgraph of Map node  n15
n3: Param
n13: Map
n14: Transpose
[('n14', 'n13')]
[('n2', 'n14')]

       Subgraph of Map node  n13
n4: Param
n6: Reduce
n11: Map
n12: Zip
[('n11', 'n6')]
[('n3', 'n12')]
[('n4', 'n12')]
[('n12', 'n11')]

       Subgraph of Reduce node  n6
n5: Mather

       Subgraph of Map node  n11
n7: Param
n9: Get
n10: Get
n8: Mather
[('n7', 'n9')]
[('n9', 'n8')]
[('n7', 'n10')]
[('n10', 'n8')]


In [23]:
program_names[1]

'mmNN'

In [24]:
a=[getmaininputs(k) for k in programs][0]

* keep stack of dearray lens
* on rearray 1) check master node 2) if map, pop dearray len 3) if reduce, pop but use len 1


In [25]:


class SV:
    def __init__(self, name,  val):
        self.label = name
        self.val = val
class Float:
    def __init__(self):
        pass
class Int:
    def __init__(self):
        pass
class Array:
    def __init__(self, length, subdata):
        self.length = length
        self.subdata = subdata
class Tuple:
    def __init__(self, subdatals = []):
        self.subdatals = subdatals
    def length():
        return len(self.subdatals)
    def add_subdata(self, subdata):
        self.subdatals.append( subdata)
class Unknown:
    def __init__(self):
        pass

In [26]:
def get_output_node(g):
    return [n for n in g['nodes'] if n.output == g['output']][0]

def get_input_nodes(g):
    return [n for n in g['nodes'] if n.input in g['inputs']]

def magic_sv():
    myopts = [10,20,30,50,5,4]
    return random.choice(myopts)

def has_ot(n):
    try:
        if n.datatype:
            return True
        else:
            return False
    except:
        return False


def cascade(program):
    g = program["graph"]
    ns = g["nodes"]
    cs = g["channels"]
    nd = dict([(n.name, n) for n in ns])
    cbdp = dict([(c.dst_port, c) for c in cs])
    svs = dict([(sv, magic_sv()) for sv in program['sizevars']])
    
    
    def form_type(s):
        op = getNextOp(s).lower()
        if op.startswith('float'):
            return Float()
        elif op.startswith('int'):
            return Int()
        elif op.startswith('array'):
            paras = getNextParams(s)
            assert(len(paras) == 2)
            sd = form_type(paras[0])
            try:
                l = int(paras[1].strip())
            except:
                l = svs[paras[1]] if 'SizeVar' not in paras[1] else magic_sv()
            return Array(l, sd)
        elif op.startswith('tuple'):
            paras = getNextParams(s)
            t = Tuple()
            [t.add_subdata(form_type(p)) for p in paras]
            return t
        else: 
            return Unknown()
    
#     in_nodes = get_input_nodes(g)
    in_types = [(k, form_type(x) )for k, x in program["inputs"]]
    
    # set types of input paramaters:
    for k, dt in in_types:
        param_nodes = [n.add_dt(dt) for n in ns if getcn(n) == 'Param' and n.label == k]
        assert(len(param_nodes) == 1)
        print("INPUT ", k, ": ", getcn(dt))
    
    dearray_stack = []
    def def_odt(node, inc_dt):
        cn = getcn(node)
        print("Datatype for ", node.name, ": ", cn, " - ", getcn(inc_dt[0]))
        if cn == "Param":
            return inc_dt # e -> e
        elif cn == "Get":
            return inc_dt.subdatals[node.idx]  #T(e, e, e, e...) -> e
        elif cn == "Transpose":
            sup_len = inc_dt.length
            sub_arr = inc_dt.subdata
            sub_len = sub_arr.length
            return Array(sub_len, Array(sup_len, sub_arr.subdata))
            # A( A(e, M), N ) -> A( A(e, N), M ) 
        elif cn == "Dearray":
            dearray_stack.append(inc_dt.length)
            return inc_dt.subdata # A(e, N) -> e
        elif cn == "Rearray":
            tmp = dearray_stack.pop()
            new_len = 1 if node.masternode == "Reduce" else tmp
            return Array(new_len, inc_dt) # e -> A(e, ?)
        elif cn == "Join":
            sup_len = inc_dt.length
            sub_arr = inc_dt.subdata
            sub_len = sub_arr.length
            return Array(sub_len * sup_len,  sub_arr.subdata)
            # A( A(e, N), M) -> A(e, N*M)
        elif cn == "Value":
            return inc_dt
        elif cn == "Zip":
            return Tuple(inc_dt)# e, r -> T(e, r)
        elif cn == "Mather":
            return inc_dt # e, e -> e    
        elif cn == "UserFun":
            return Unknown() # e -> b
       
    visited = []
    def get_odt(node):
        print(node.name, ": ", getcn(node))
        
        # base case: Params should already be filled in
        if has_ot(node):
            return node.datatype
        if node.name in visited:
            return Unknown()
        visited.append(node.name)
        
        input_nodes = [get_connected_node(g, i.name) for i in getin(node)]
        input_ports = getin(node)
        input_channels = [cbdp[p.name] for p in input_ports]
        input_nodes2 = [nd[c.src_act] for c in input_channels]
        assert(input_nodes == input_nodes2)
        print("inputs: ", [n.name for n in input_nodes])
        print()
        
        # get outgoing datatype for inputs 
        input_datatypes = [get_odt(n) for n in input_nodes]
        if len(input_datatypes) == 1:
            input_datatypes = input_datatypes[0]
        node_odt = def_odt(node, input_datatypes)
        node.add_dt(node_odt)
        return(node_odt)
    
    root = get_output_node(g)
    
    get_odt(root)

In [27]:
h = unpack_program(programs[2])

In [48]:
def get_output_node(g):
    return [n for n in g['nodes'] if n.output == g['output']][0]

def get_input_nodes(g):
    return [n for n in g['nodes'] if n.input in g['inputs']]

def magic_sv():
    myopts = [10,20,30,50,5,4]
    return random.choice(myopts)

def has_ot(n):
    try:
        n.datatype
        return True
    except:
        return False


def cascade(program):
    g = program["graph"]
    ns = g["nodes"]
    cs = g["channels"]
    nd = dict([(n.name, n) for n in ns])
    cbdp = dict([(c.src_port, c) for c in cs])
    svs = dict([(sv, magic_sv()) for sv in program['sizevars']])
    
    
    def form_type(s):
        op = getNextOp(s).lower()
        if op.startswith('float'):
            return Float()
        elif op.startswith('int'):
            return Int()
        elif op.startswith('array'):
            paras = getNextParams(s)
            assert(len(paras) == 2)
            sd = form_type(paras[0])
            try:
                l = int(paras[1].strip())
            except:
                l = svs[paras[1]] if 'SizeVar' not in paras[1] else magic_sv()
            return Array(l, sd)
        elif op.startswith('tuple'):
            paras = getNextParams(s)
            t = Tuple()
            [t.add_subdata(form_type(p)) for p in paras]
            return t
        else: 
            return Unknown()
    
#     in_nodes = get_input_nodes(g)
    in_types = [(k, form_type(x) )for k, x in program["inputs"]]
    
    
    dearray_stack = []
    def def_odt(node, inc_dt):
        cn = getcn(node)
        print("Datatype for ", node.name, ": ", cn, " - ", getcn(inc_dt))
        if cn == "Param":
            return inc_dt # e -> e
        elif cn == "Get":
            return inc_dt.subdatals[node.idx]  #T(e, e, e, e...) -> e
        elif cn == "Transpose":
            sup_len = inc_dt.length
            sub_arr = inc_dt.subdata
            sub_len = sub_arr.length
            return Array(sub_len, Array(sup_len, sub_arr.subdata))
            # A( A(e, M), N ) -> A( A(e, N), M ) 
        elif cn == "Dearray":
            dearray_stack.append(inc_dt.length)
            return inc_dt.subdata # A(e, N) -> e
        elif cn == "Rearray":
            tmp = dearray_stack.pop()
            new_len = 1 if node.masternode == "Reduce" else tmp
            return Array(new_len, inc_dt) # e -> A(e, ?)
        elif cn == "Join":
            sup_len = inc_dt.length
            sub_arr = inc_dt.subdata
            sub_len = sub_arr.length
            return Array(sub_len * sup_len,  sub_arr.subdata)
            # A( A(e, N), M) -> A(e, N*M)
        elif cn == "Value":
            return inc_dt
        elif cn == "Zip":
            return Tuple(inc_dt)# e, r -> T(e, r)
        elif cn == "Mather":
            return inc_dt # e, e -> e    
        elif cn == "UserFun":
            return Unknown() # e -> b
    
    
    visited = []
    def push(node):
        print("Pushing: ", node.name, getcn(node))
        if node.name in visited or node == get_output_node(g):
            return
        visited.append(node.name)
        print(node.get_dt())
        assert(node.has_dt())
        my_dt = node.get_dt()
        
        print("Pushing: ", node.name, " (",my_dt,")")
        output_nodes = [get_connected_node(g, out.name) for out in getout(node)]
        output_ports = getout(node)
        output_channels = [cbdp[p.name] for p in output_ports]
        output_nodes2 = [nd[c.dst_act] for c in output_channels]
        assert(output_nodes == output_nodes2)
        print("outputs: ", [n.name for n in output_nodes])
        print()
        
        for out_node in output_nodes:
            if getcn(out_node) != 'Zip':
                new_dt = def_odt(out_node, my_dt)
                out_node.add_dt(new_dt)
                push(out_node)
            else:
                if all( [get_connected_node(g, inc.name).has_dt() for inc in getin(node)]):
                    my_dts = [get_connected_node(g, inc.name).datatype for inc in getin(node)]
                    new_dt = def_odt(out_node, my_dts)
                    out_node.add_dt(new_dt)
                    print(new_dt, out_node, out_node.get_dt())
                    push(out_node)
    
    
    # set types of input paramaters:
    for k, dt in in_types:
        param_nodes = [n for n in ns if getcn(n) == 'Param' and n.label == k]
        assert(len(param_nodes) == 1)
        param_node = param_nodes[0]
        print("INPUT ", param_node.name)
        param_node.add_dt(dt)
        print(param_node.get_dt())
        push(param_node)
        print("INPUT ", k, ": ", getcn(dt))

In [49]:
h = unpack_program(programs[1])
explode(h['graph'])
cascade(h)

INPUT  n1
Pushing:  n1 Param
Pushing:  n1  ( <__main__.Array object at 0x1062bb860> )
outputs:  ['n15XDA']

Datatype for  n15XDA :  Dearray  -  Array
Pushing:  n15XDA Dearray
Pushing:  n15XDA  ( <__main__.Array object at 0x106287d30> )
outputs:  ['n3']

Datatype for  n3 :  Param  -  Array
Pushing:  n3 Param
Pushing:  n3  ( <__main__.Array object at 0x106287d30> )
outputs:  ['n12']

Datatype for  n12 :  Zip  -  list
<__main__.Tuple object at 0x106287080> <actor name='n12' type='Zip'>
<port type='out' name='n12_out' rate='1'/>
<port type='in' name='n12_x0_in' rate='1'/>
<port type='in' name='n12_x1_in' rate='1'/>
</actor> <__main__.Tuple object at 0x106287080>
Pushing:  n12 Zip
Pushing:  n12  ( <__main__.Tuple object at 0x106287080> )
outputs:  ['n11XDA']

Datatype for  n11XDA :  Dearray  -  Tuple


AttributeError: 'Tuple' object has no attribute 'subdata'

In [ ]:
gg = h['graph']
gg['nodes']

In [ ]:

allsvs = set(flatten([p['sizevars'] for p in parsed_programs]))

In [ ]:
magic_sv()

In [ ]:
def get_output_node(g):
    return [n for n in g['nodes'] if n.output == g['output']][0]

def get_input_nodes(g):
    return [n for n in g['nodes'] if n.input in g['inputs']]

In [ ]:
get_input_nodes(h['graph'])[0].label

In [ ]:
print(get_output_node)
printls(h['graph']['inputs'])

In [ ]:
h['inputs']